In [1]:
import os
import json
import spacy
import wptools
import requests
import wikipedia
import pandas as pd
from bs4 import BeautifulSoup

def fetch_nobel_laureates(url, category='not set'):
  response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
  soup = BeautifulSoup(response.content, 'html.parser')

  # Find the table containing the data
  tables = soup.find_all('table', {'class': 'wikitable'})

  nobel_laureates_df = pd.read_html(str(tables))
  nobel_laureates_df = nobel_laureates_df[0]
  nobel_laureates_df.columns = nobel_laureates_df.columns.str.lower() # Lowercase column names
  nobel_laureates_df = nobel_laureates_df.drop(columns=['image', 'rationale[c]', 'ref'])
  nobel_laureates_df = nobel_laureates_df.rename(columns={'laureate[a]': 'name', 'country[b]': 'country'})
  nobel_laureates_df['name'] = nobel_laureates_df['name'].str.replace(r'\s*\([^()]*\)', '', regex=True)
  # Drop duplicate name rows
  nobel_laureates_df = nobel_laureates_df.drop_duplicates(subset=['name'], keep='first')
  # Drop rows with missing values in the name column (e.g. empty rows)
  nobel_laureates_df = nobel_laureates_df.dropna(subset=['name'])
  # Excluding hard to parse rows
  nobel_laureates_df = nobel_laureates_df[~nobel_laureates_df['name'].str.contains('Not awarded', case=False)]
  nobel_laureates_df = nobel_laureates_df[~nobel_laureates_df['name'].str.contains('Skłodowska', case=False)]
  nobel_laureates_df = nobel_laureates_df[~nobel_laureates_df['name'].str.contains(', 7th', case=False)]
  nobel_laureates_df = nobel_laureates_df[~nobel_laureates_df['name'].str.contains("'ichirō", case=False)]
  nobel_laureates_df = nobel_laureates_df[~nobel_laureates_df['name'].str.contains("van 't Hoff", case=False)]
  # Add a column for the category
  nobel_laureates_df['category'] = category
  return nobel_laureates_df.head(100)


In [2]:
# Create function to fetch biography of laureates from Wikipedia
def fetch_biography(name):
  search = wikipedia.search(name)[0]
  try:
    page = wikipedia.page(search)
    full_content = page.content # we are getting all the content of the page for now
  except:
    # suggestions
    wp_page = wptools.page(name)
    query = wp_page.get_query()
    page_id = query.data["pageid"]
    page = wikipedia.page(pageid=page_id)
    full_content = page.content
  return full_content

In [3]:
def fetch_rdf_triples(name):
    # Replace spaces with underscores
    name = name.replace(" ", "_")
    query = f"""
    SELECT DISTINCT ?subject ?predicate ?object
    WHERE {{
      {{
        ?subject ?predicate dbr:{name} .
        ?subject ?predicate ?object .
      }}
      UNION
      {{
        ?object ?predicate dbr:{name} .
        ?subject ?predicate ?object .
      }}
    }}
    LIMIT 100
    """




    url = "https://dbpedia.org/sparql"
    headers = {
        "Accept": "application/sparql-results+json"
    }
    params = {
        "query": query,
        "format": "json"
    }

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  # Raises an HTTPError for bad responses
        return response.json()['results']['bindings']
    except requests.exceptions.RequestException as e:
        return {"error": str(e)}

In [4]:
# if the data directory does not exist, create it
if not os.path.exists(r'..\..\Web Scraped Corpus\physics_nobel_laureate'):
    os.makedirs(r'..\..\Web Scraped Corpus\physics_nobel_laureate')

if not os.path.exists(r'..\..\Web Scraped Corpus\chemistry_nobel_laureate'):
    os.makedirs(r'..\..\Web Scraped Corpus\chemistry_nobel_laureate')

physics_nobel_laureate = fetch_nobel_laureates('https://en.wikipedia.org/wiki/List_of_Nobel_laureates_in_Physics', 'Physics')

# Apply the fetch_biography function to the name column of the DataFrame
physics_nobel_laureate['biography'] = physics_nobel_laureate['name'].apply(fetch_biography)

# Save each laureate's data to a /data/name_physics_nobel_laureate.txt file
for index, row in physics_nobel_laureate.iterrows():
    name = row['name']
    row_string = '\n'.join([f'{key}: {value}' for key, value in row.items()])
    # Save the data to a file

    with open(f'..\..Web Scraped Corpus\physics_nobel_laureate\{name.replace(" ", "__")}.txt', 'w', encoding="utf-8") as f:
        f.write(row_string)
    rdf_triples = fetch_rdf_triples(name) # Fetch RDF triples
    # Save the RDF triples to a json file
    with open(f'..\..Web Scraped Corpus\physics_nobel_laureate\{name.replace(" ", "__")}.json', 'w', encoding="utf-8") as f:
        json.dump(rdf_triples, f, indent=4)

# print(len(physics_nobel_laureate))
# print(physics_nobel_laureate.head(10))

C:\Users\alber\AppData\Local\Temp\ipykernel_17464\1249743613.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  nobel_laureates_df = pd.read_html(str(tables))
Marie Curie (en) data
{
  WARNINGS: <dict(2)> main, extracts
  aliases: <list(4)> Maria Salomea Skłodowska, Maria Skłodowska-Cu...
  assessments: <dict(10)> Medicine, Military history, Biography, P...
  description: Polish and French physicist and chemist (1867–1934)
  extext: <str(2778)> **Maria Salomea Skłodowska-Curie** (Polish: ...
  extract: <str(3406)> <p class="mw-empty-elt"></p><p><b>Maria Sal...
  image: <list(4)> {'kind': 'query-pageimage', 'file': 'File:Marie...
  label: Marie Curie
  length: 109,351
  links: <list(829)> 1903 Nobel Memorial Prize in Economic Science...
  modified: <dict(1)> page
  pageid: 20408
  random: List of WTA Tour top-level tournament singles champions
  redirects: <list

In [26]:
chemistry_nobel_laureate = fetch_nobel_laureates('https://en.wikipedia.org/wiki/List_of_Nobel_laureates_in_Chemistry', 'Chemistry')

# Apply the fetch_biography function to the name column of the DataFrame
chemistry_nobel_laureate['biography'] = chemistry_nobel_laureate['name'].apply(fetch_biography)

# Save each laureate's data to a /data/name_chemistry_nobel_laureate.txt file
for index, row in chemistry_nobel_laureate.iterrows():
    name = row['name']



    row_string = '\n'.join([f'{key}: {value}' for key, value in row.items()])
    # Save the data to a file
    with open(f'..\..\Web Scraped Corpus\chemistry_nobel_laureate\{name.replace(" ", "__")}.txt', 'w', encoding="utf-8") as f:
        f.write(row_string)
    rdf_triples = fetch_rdf_triples(name) # Fetch RDF triples
    # Save the RDF triples to a json file
    with open(f'..\..\Web Scraped Corpus\chemistry_nobel_laureate\{name.replace(" ", "__")}.json', 'w', encoding="utf-8") as f:
        json.dump(rdf_triples, f, indent=4)

# print(len(chemistry_nobel_laureate))
# print(chemistry_nobel_laureate.head(20))

/var/folders/6t/344bnnqx6s76nkzddhpzrfwm0000gn/T/ipykernel_81161/1832671345.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  nobel_laureates_df = pd.read_html(str(tables))
en.wikipedia.org (query) Sir William Ramsay
en.wikipedia.org (imageinfo) File:William Ramsay.jpg
William Ramsay (en) data
{
  WARNINGS: <dict(1)> extracts
  aliases: <list(1)> Sir William Ramsay
  assessments: <dict(3)> Biography, Scotland, Chemistry
  description: Scottish chemist
  extext: <str(600)> **Sir William Ramsay** (; 2 October 1852 – 23...
  extract: <str(660)> <p class="mw-empty-elt"></p><p><b>Sir Willia...
  image: <list(2)> {'kind': 'query-pageimage', 'file': 'File:Willi...
  label: William Ramsay
  length: 17,024
  links: <list(299)> 1902 Coronation Honours, 1904 Nobel Memorial ...
  modified: <dict(1)> page
  pageid: 48187
  random: List of buildings and structures in Burund

In [27]:
physics_and_chemistry_nobel_laureate = pd.concat([physics_nobel_laureate, chemistry_nobel_laureate], axis=0)

In [28]:
nlp = spacy.load('en_core_web_sm')

def clean_text_spacy(text):
    doc = nlp(text)
    # Lemmatize the text and remove the stopwords and non-alphabetic characters
    words = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    return ' '.join(words)

# Apply the clean_text_spacy function to the biography column
physics_and_chemistry_nobel_laureate['clean_biography'] = physics_and_chemistry_nobel_laureate['biography'].apply(clean_text_spacy)

In [29]:
physics_and_chemistry_nobel_laureate.head(10)

,year,name,country,category,biography,clean_biography
0,1901,Wilhelm Röntgen,German Empire,Physics,Wilhelm Conrad Röntgen (; German pronunciation...,Wilhelm Conrad Röntgen german pronunciation ˈv...
1,1902,Hendrik Lorentz,Netherlands,Physics,Hendrik Antoon Lorentz (; 18 July 1853 – 4 Feb...,Hendrik Antoon Lorentz July February dutch phy...
2,1902,Pieter Zeeman,Netherlands,Physics,Pieter Zeeman (Dutch: [ˈzeːmɑn]; 25 May 1865 –...,Pieter Zeeman Dutch ˈzeːmɑn October dutch phys...
3,1903,Henri Becquerel,France,Physics,Antoine Henri Becquerel (; French pronunciatio...,Antoine Henri Becquerel french pronunciation b...
4,1903,Pierre Curie,France,Physics,"Pierre Curie ( KURE-ee, French: [pjɛʁ kyʁi]; 1...",Pierre Curie KURE ee French pjɛʁ kyʁi April fr...
5,1903,Marie Curie,Poland ( Russian Empire) France,Physics,Maria Salomea Skłodowska-Curie (Polish: [ˈmarj...,Maria Salomea Skłodowska Curie Polish ˈmarja s...
6,1904,Lord Rayleigh,United Kingdom,Physics,"John William Strutt, 3rd Baron Rayleigh, (; 1...",John William Strutt Baron Rayleigh November Ju...
7,1905,Philipp Lenard,German Empire,Physics,Philipp Eduard Anton von Lenard (German pronun...,Philipp Eduard Anton von Lenard german pronunc...
8,1906,J. J. Thomson,United Kingdom,Physics,Sir Joseph John Thomson (18 December 1856 – 3...,Sir Joseph John Thomson December August britis...
9,1907,Albert A. Michelson,United States,Physics,Albert Abraham Michelson FFRS FRSE (surname pr...,Albert Abraham Michelson FFRS FRSE surname pro...


In [30]:
physics_and_chemistry_nobel_laureate.to_csv(r"..\..\Web Scraped Corpus\physics_and_chemistry_nobel_laureate.csv", index=False)